In [1]:
from glob import glob
import json
import re
import pandas as pd

In [2]:
zveza_all = []
for x in glob("./data/scraped/kzp_in_ko/*.json"):
    with open(x, "r") as f:
        data = json.load(f)

    zveza_all.append((data["zveza"], data["meta"]["Evidenčna številka"]))


In [3]:
set([y for x in zveza_all for y in x[0].keys()])

{'',
 'EU - Direktive, Uredbe, Sklepi / Odločbe, Sporazumi, Pravila',
 'Konvencije, Deklaracije Resolucije',
 'Mednarodne Pogodbe',
 'Podzakonski akti / Vsi drugi akti',
 'Predpisi drugih držav',
 'RS - Ustava, Zakoni, Sporazumi, Pogodbe',
 'Without category'}

In [4]:
zveza_unformatted = set([(*y, x[1]) for x in zveza_all for y in x[0].items()]) - set(
    [""]
)
zveza_unformatted = pd.DataFrame(
    zveza_unformatted, columns=["category", "zveza", "evidencna_stevilka"]
)
zveza_unformatted

,category,zveza,evidencna_stevilka
0,Without category,"ZKP člen 354, 371, 371/1-9.\n \t\t KZ čl...",VS2005565
1,Without category,"ZKP člen 344, 344/1, 372, 372/1. ZOPOKD člen 4.",VS2007979
2,Without category,"ZKP člen 344, 359, 359/1-1, 364, 364/4, 371, 3...",VS2006918
3,Without category,"ZKP člen 372, 372-1, 420, 420/2.\n \t\t ...",VS2006946
4,"RS - Ustava, Zakoni, Sporazumi, Pogodbe",Zakon o kazenskem postopku (1994) - ZKP - čle...,VS00010030
...,...,...,...
3068,"RS - Ustava, Zakoni, Sporazumi, Pogodbe","Kazenski zakonik (2008) - KZ-1 - člen 308, 30...",VS00059159
3069,Without category,"ZKP člen 257, 258, 371, 371/2.\n \t\t KZ...",VS2005487
3070,Without category,"ZKP člen 371, 371/2.\n \t\t URS člen 22.",VS2005506
3071,Without category,"ZKP člen 18, 329, 371, 420, 420/2.",VS2006750


# ZAKONI RS

In [5]:
def split_RS(zveza):
    zveza = re.sub(r"[\n\t\r]", "", zveza)

    split_phrase = "#$%^"
    pattern = r"[A-Z][\w\s]+\(\d+\)"
    # zveza_splitters = [".", "Konvencija ", "Direktiva ", "Uredba ", "Pravilnik "]

    matches = re.findall(pattern, zveza)
    matches.append("Zakon o")

    for match in matches:
        zveza = zveza.replace(match, f"{split_phrase}{match}")

    return [x.strip() for x in zveza.split(split_phrase) if x]


def _format_clen_from_docs(clen):
    if clen.isdigit():
        return f"{clen}. člen"
    digits = "".join([x for x in clen if x.isdigit()])
    alphas = "".join([x for x in clen if x.isalpha()])
    return f"{digits}.{alphas} člen"


formatted_data = []
for _val in zveza_unformatted.loc[
    zveza_unformatted["category"] == "RS - Ustava, Zakoni, Sporazumi, Pogodbe",
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = _val
    for z in split_RS(zveza):
        try:
            _tmp = z.split(" - ")
            if len(_tmp) == 3:
                title, abbr, clens_unformatted = _tmp
            else:
                title, clens_unformatted = _tmp
                abbr = ""

            title, abbr, clens_unformatted = (
                title.strip(),
                abbr.strip(),
                clens_unformatted.strip(),
            )
            title = re.sub(r"\(\d+\)", "", title)
        except:
            print("ERROR split: ", z)
            continue

        try:
            zveza_true = []
            for clen in clens_unformatted.split(","):
                clen = clen.strip()
                clen = clen.split("/")[0]
                clen = clen.split("-")[0]
                if _split := clen.split(" "):
                    clen = [x for x in _split if not all(y.isalpha() for y in x)][0]
                zveza_true.append(clen)

            zveza_true = sorted(
                [f"{_format_clen_from_docs(x)} {abbr}" for x in list(set(zveza_true))],
                key=lambda x: int(x.split(".")[0]),
            )
            formatted_data.append(
                {
                    "title": title,
                    "abbr": abbr,
                    "zveza": zveza_true,
                    "evidencna_stevilka": evidencna_stevilka,
                }
            )
        except:
            print("ERROR formatting: ", zveza)
            continue


ERROR split:  Zakon o ratifikaciji Evropske konvencije o izročitvi,
ERROR split:  Zakon o kazenskem postopku (1994) - ZKP  - člen 18, 18/2, 371, 371/1-8, 420, 420/5. Obligacijski zakonik (uradno prečiščeno besedilo) (2007) - OZ-UPB1  - člen 731, 731/1.
ERROR formatting:  Zakon o kazenskem postopku (1994) - ZKP  - člen 201, 201/1, 201/1-3,  Kazenski zakonik (1994) - KZ  - člen 86
ERROR formatting:  Zakon o kazenskem postopku (1994) - ZKP  - člen 371, 371/1, 371/1-11, 372, 372-1, 372-4,  Kazenski zakonik (1994) - KZ  - člen 235, 235/1, 244, 244/1
ERROR formatting:  Kazenski zakonik (2008) - KZ-1  - člen 240, 54,
ERROR formatting:  Kazenski zakonik (2008) - KZ-1  - člen 20, 20/2, 38, 74, 75, 75/5, 175, 175/1, 175/3, 245, 245/1, 245/2, 245/3, 245/4, Ustava Republike Slovenije (1991) - URS  - člen 29, 29/3 Zakon o kazenskem postopku (1994) - ZKP  - člen 18, 351, 351/1, 364, 364/7, 395, 395/1, 501
ERROR formatting:  Zakon o kazenskem postopku (1994) - ZKP  - člen 42, 144, 294, 295, 371, 371/

In [6]:
RS_cleaned_data = pd.DataFrame(formatted_data)
RS_cleaned_data

,title,abbr,zveza,evidencna_stevilka
0,Zakon o kazenskem postopku,ZKP,"[39. člen ZKP, 40. člen ZKP, 41. člen ZKP, 371...",VS00010030
1,Ustava Republike Slovenije,URS,[23. člen URS],VS00010030
2,Zakon o kazenskem postopku,ZKP,[272. člen ZKP],VS00024691
3,Ustava Republike Slovenije,URS,"[22. člen URS, 29. člen URS]",VS00024691
4,Zakon o kazenskem postopku,ZKP,"[8. člen ZKP, 368. člen ZKP]",VS00032394
...,...,...,...,...
1663,Zakon o kazenskem postopku,ZKP,"[18. člen ZKP, 289. člen ZKP, 329. člen ZKP]",VS00077953
1664,Kazenski zakonik,KZ-1,[308. člen KZ-1],VS00059159
1665,Zakon o kazenskem postopku,ZKP,"[10. člen ZKP, 371. člen ZKP, 385. člen ZKP]",VS00059159
1666,Ustava Republike Slovenije,URS,[29. člen URS],VS00059159


# WITHOUT CATEGORY

In [7]:
def split_uncatagorized(zveza):
    zveza = re.sub(r"[\n\t\r]", "", zveza)

    split_phrase = "#$%^"
    pattern = r"[\w\d-]+ člen"

    matches = re.findall(pattern, zveza)

    for match in matches:
        zveza = zveza.replace(match, f"{split_phrase}{match}")

    return [x.strip() for x in zveza.split(split_phrase) if x]


formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"] == "Without category",
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    zveza = split_uncatagorized(zveza)

    for z in zveza:
        pattern = r"[\w\d-]+ člen"
        matches = re.findall(pattern, z)

        if len(matches) != 1:
            print("ERROR MATCH: ", z)
            continue

        abbr = matches[0].replace("člen", "").strip()
        z = z.replace(matches[0], "")
        zveza_split = z.split(",")
        zveza_split = [
            x.strip().strip(".").split("/")[0].split("-")[0] for x in zveza_split if x
        ]

        try:
            zveza_true = sorted(
                [f"{_format_clen_from_docs(x)} {abbr}" for x in list(set(zveza_split))],
                key=lambda x: int(x.split(".")[0]),
            )
        except:
            print("ERROR SPLIT: ", zveza_split)
            continue

        formatted_data.append(
            {
                "title": "Without category",
                "abbr": abbr,
                "zveza": zveza_true,
                "evidencna_stevilka": evidencna_stevilka,
            }
        )
WC_cleaned_data = pd.DataFrame(formatted_data)
WC_cleaned_data


ERROR SPLIT:  ['201', '201', '424', '424', '525. Konvencija o izročitvi storilcev kaznivih dejanj med Kraljevino Srbov', 'Hrvatov in Slovencev in Republiko']
ERROR MATCH:  Sodni
ERROR MATCH:  ZKP 371, 371/2, 395, 395/1.
ERROR MATCH:  ZKP 35, 35/1.
ERROR SPLIT:  ['92', '92', 'ki opravljajo vročanje v pravdnem        postopku.        Pravilnik o delovanju oseb', 'ki opravljajo vročanje v kazenskem        postopku']
ERROR SPLIT:  ['205', '205', '', '207', '207', '272', '272', '420', '420', '420']
ERROR MATCH:  ZKP, 149b, 149b/1, 320, 325, 325/1, 344, 344/1, 355, 355/2, 371, 371/1-8, 420, 420/2.
ERROR MATCH:  ZKP, člen 201, 201/1-1, 201/1-3, 272.


,title,abbr,zveza,evidencna_stevilka
0,Without category,ZKP,"[354. člen ZKP, 371. člen ZKP]",VS2005565
1,Without category,KZ,"[169. člen KZ, 171. člen KZ, 172. člen KZ]",VS2005565
2,Without category,ZKP,"[344. člen ZKP, 372. člen ZKP]",VS2007979
3,Without category,ZOPOKD,[4. člen ZOPOKD],VS2007979
4,Without category,ZKP,"[344. člen ZKP, 359. člen ZKP, 364. člen ZKP, ...",VS2006918
...,...,...,...,...
2938,Without category,ZKP,"[257. člen ZKP, 258. člen ZKP, 371. člen ZKP]",VS2005487
2939,Without category,KZ,[240. člen KZ],VS2005487
2940,Without category,ZKP,[371. člen ZKP],VS2005506
2941,Without category,URS,[22. člen URS],VS2005506


EU

In [8]:
eu_types = [
    "EU - Direktive, Uredbe, Sklepi / Odločbe, Sporazumi, Pravila",
]

formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"].isin(eu_types),
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    for _direktiva in [f"Direktiva{x}" for x in zveza.split("Direktiva") if x.strip()]:
        title, clens_unformatted = _direktiva.split(" - ")
        pattern = r"\d{4}/\d{1,2}/EU"
        matches = re.findall(pattern, title)

        if len(matches) != 1:
            print("ERROR MATCH: ", z)
            continue
        abbr = matches[0]

        clens_unformatted = clens_unformatted.replace("člen", "")
        clens_split = [
            x.strip().strip(".").split("/")[0].split("-")[0]
            for x in clens_unformatted.split(",")
        ]

        try:
            zveza_true = sorted(
                [f"{_format_clen_from_docs(x)} {abbr}" for x in list(set(clens_split))],
                key=lambda x: int(x.split(".")[0]),
            )
        except:
            print("ERROR SPLIT: ", clens_split)
            continue

        formatted_data.append(
            {
                "title": title,
                "abbr": abbr,
                "zveza": zveza_true,
                "evidencna_stevilka": evidencna_stevilka,
            }
        )
EU_cleaned_data = pd.DataFrame(formatted_data)
EU_cleaned_data


,title,abbr,zveza,evidencna_stevilka
0,Direktiva 2012/13/EU Evropskega parlamenta in ...,2012/13/EU,[6. člen 2012/13/EU],VS00080627
1,Direktiva 2013/48/EU Evropskega parlamenta in ...,2013/48/EU,"[2. člen 2013/48/EU, 3. člen 2013/48/EU]",VS00077972
2,Direktiva 2012/13/EU Evropskega parlamenta in ...,2012/13/EU,"[2. člen 2012/13/EU, 3. člen 2012/13/EU, 6. čl...",VS00077972
3,Direktiva 2010/64/EU Evropskega parlamenta in ...,2010/64/EU,[3. člen 2010/64/EU],VS00032394
4,Direktiva 2012/13/EU Evropskega parlamenta in ...,2012/13/EU,[6. člen 2012/13/EU],VS00081568
5,Direktiva 2012/29/EU Evropskega parlamenta in ...,2012/29/EU,[21. člen 2012/29/EU],VS00004246


OTHER

In [9]:
other_types = [
    "Konvencije, Deklaracije Resolucije",
    "Mednarodne Pogodbe",
    "Podzakonski akti / Vsi drugi akti",
    "Predpisi drugih držav",
]

formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"].isin(other_types),
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    print(zveza)


Pravilnik za organizacijo, poslovanje in postopek brigadnih in odrednih vojaških sodišč (1943) - člen 1Odlok o formiranju izrednega vojaškega sodišča (1943) - člen 1Uredba o vojaškem kazenskem sodstvu (1944) - člen 2, 2/1Uredba o vojaških sodiščih (1944) - člen 1Odlok o odpravi in razveljavljenju vseh pravnih predpisov, izdanih med okupacijo, po okupatorjih in njihovih pomagačih, o veljavnosti odločb izdanih v tej dobi, o odpravi pravnih predpisov, ki so veljali v trenutku okupacije po sovražniku (1945) - člen 1, 2, 6
Uredba o razvrstitvi prepovedanih drog (2019) - priloga Skupina II/112
Evropska Konvencija o medsebojni pravni pomoči v kazenskih zadevah - člen 3, 3/1.
Konvencija o varstvu človekovih pravic in temeljnih svoboščin (EKČP)  - člen 6, 8
Sodni red - člen 77, 77/2.
Sodni red (2016) - člen 244.
Konvencija ZN o otrokovih pravicah - člen 1.Evropska konvencija o uresničevanju otrokovih pravic - člen 1, 1/1.
Konvencija o varstvu človekovih pravic in temeljnih svoboščin (EKČP) - čl

In [11]:
merged_data = pd.concat([RS_cleaned_data, WC_cleaned_data, EU_cleaned_data])
merged_data["title"] = merged_data["title"].str.replace("Without category", "")
merged_data.reset_index(drop=True).to_parquet(
    "./data/mappings/legislation_mappings_from_kzp_scraped.parquet"
)